# 🎵 A2SB: Audio-to-Audio Schrödinger Bridge - Kaggle Edition

**High-Quality Audio Restoration with NVIDIA A2SB**

This notebook includes **everything** you need: model download, setup, and Gradio web interface!

## ⚠️ IMPORTANT: GPU Required

**This notebook requires Kaggle GPU acceleration!**

### GPU Requirements:
- 🎯 **GPU Memory**: Requires 15GB+ GPU VRAM (P100 or T4 x2)
- ⏱️ **Processing Time**: Long audio files need extended runtime
- 💾 **RAM**: Minimum 25GB system RAM recommended
- 🚀 **Performance**: Better GPUs for faster processing

### How to Enable GPU on Kaggle:
1. Click **Settings** (right sidebar)
2. Under **Accelerator**, select **GPU P100** or **GPU T4 x2**
3. Click **Save**

### Kaggle Advantages:
- ✅ Free GPU access (30 hours/week)
- ✅ P100 GPU with 16GB VRAM
- ✅ 30GB RAM
- ✅ Persistent storage
- ✅ No subscription required

---

## 🌟 Features
- ✅ 44.1kHz high-resolution music restoration
- ✅ Bandwidth extension (high-frequency prediction)
- ✅ Audio inpainting (reconstruct missing segments)
- ✅ Support for long audio files (hours)
- ✅ End-to-end, no vocoder required
- ✅ **Gradio Web Interface** - User-friendly UI

## 📚 Resources
- 📄 [Paper](https://arxiv.org/abs/2501.11311)
- 💻 [GitHub Repository](https://github.com/test4373/diffusion-audio-restoration-colab-Kaggle-.git)
- 🎬 [Original NVIDIA Demo](https://research.nvidia.com/labs/adlr/A2SB/)
- 🤗 [Models](https://huggingface.co/nvidia/audio_to_audio_schrodinger_bridge)

---

**Usage:** Run cells in order. The last cell will launch the Gradio interface!

## 🔍 Step 0: Check Your Kaggle Environment

**Run this cell to verify your GPU and system status.**

In [ ]:
import subprocess
import sys
import os

print("="*60)
print("🔍 CHECKING KAGGLE ENVIRONMENT")
print("="*60)

# Check GPU
try:
    gpu_info = subprocess.check_output(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], text=True)
    gpu_name, gpu_memory = gpu_info.strip().split(', ')
    gpu_memory_gb = int(gpu_memory.split()[0]) / 1024
    
    print(f"\n✓ GPU Detected: {gpu_name}")
    print(f"✓ GPU Memory: {gpu_memory_gb:.1f} GB")
    
    # Check if sufficient
    if gpu_memory_gb < 14:
        print("\n" + "="*60)
        print("⚠️  WARNING: INSUFFICIENT GPU MEMORY")
        print("="*60)
        print(f"Your GPU has {gpu_memory_gb:.1f} GB memory.")
        print("This notebook requires at least 15GB GPU memory.")
        print("\n🔴 RECOMMENDATION: Enable P100 GPU in Kaggle Settings!")
        print("\nYou may experience:")
        print("  - Out of Memory errors")
        print("  - Failed inference")
        print("  - Slow processing")
        print("\n👉 Enable GPU: Settings > Accelerator > GPU P100")
        print("="*60)
    else:
        print("\n✅ GPU memory is sufficient!")
        if 'P100' in gpu_name:
            print("🎉 You have a P100 GPU! Perfect for this notebook!")
        elif 'T4' in gpu_name:
            print("✓ T4 GPU detected - Good for this notebook")
        elif 'V100' in gpu_name:
            print("🎉 You have a V100 GPU! Excellent performance!")
            
except Exception as e:
    print("\n❌ ERROR: No GPU detected!")
    print("\nPlease enable GPU:")
    print("1. Click 'Settings' in the right sidebar")
    print("2. Under 'Accelerator', select 'GPU P100' or 'GPU T4 x2'")
    print("3. Click 'Save'")
    print("4. Restart the notebook")
    sys.exit(1)

# Check RAM
try:
    with open('/proc/meminfo', 'r') as f:
        meminfo = f.read()
    mem_total = int([line for line in meminfo.split('\n') if 'MemTotal' in line][0].split()[1]) / 1024 / 1024
    print(f"\n✓ System RAM: {mem_total:.1f} GB")
    
    if mem_total < 20:
        print("⚠️  Low RAM detected. This may cause issues with large audio files.")
    else:
        print("✅ RAM is sufficient!")
except:
    pass

# Check Kaggle working directory
print(f"\n✓ Working Directory: {os.getcwd()}")
print(f"✓ Kaggle Input: /kaggle/input/")
print(f"✓ Kaggle Working: /kaggle/working/")

print("\n" + "="*60)
print("✅ Environment check complete!")
print("="*60)
print("\nKaggle provides free GPU access with:")
print("  • 30 hours GPU time per week")
print("  • P100 GPU (16GB VRAM)")
print("  • 30GB RAM")
print("  • 73GB disk space")
print("="*60)

## 📦 1. Setup and Dependencies

**This will take 5-10 minutes. Please be patient!**

In [ ]:
# Clone the optimized repository
print("📥 Cloning repository...\n")

# Change to /kaggle/working directory (writable)
import os
os.chdir('/kaggle/working')

# Remove if exists
!rm -rf diffusion-audio-restoration-colab-Kaggle-

# Clone repository
!git clone https://github.com/test4373/diffusion-audio-restoration-colab-Kaggle-.git
os.chdir('diffusion-audio-restoration-colab-Kaggle-')

print(f"\n✅ Repository cloned successfully!")
print(f"✓ Current directory: {os.getcwd()}")

In [ ]:
# Install required libraries - FIXED for torchaudio compatibility
print("📦 Installing dependencies...\n")
print("⏱️  This will take 5-10 minutes. Please wait...\n")

# Step 1: Fix torchaudio compatibility issue
print("🔧 Fixing torchaudio compatibility...")
!pip uninstall -y torchaudio 2>/dev/null || true
import torch
torch_version = torch.__version__.split('+')[0]
print(f"✓ Detected PyTorch version: {torch_version}")

# Install matching torchaudio
if torch_version.startswith('2.6'):
    !pip install -q torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu118
elif torch_version.startswith('2.5'):
    !pip install -q torchaudio==2.5.0 --index-url https://download.pytorch.org/whl/cu118
elif torch_version.startswith('2.4'):
    !pip install -q torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu118
else:
    !pip install -q torchaudio --index-url https://download.pytorch.org/whl/cu118

# Step 2: Install other packages
print("\n📥 Installing required packages...")
!pip install -q lightning "pytorch-lightning>=2.0.0"
!pip install -q librosa soundfile
!pip install -q einops gradio "jsonargparse[signatures]>=4.0.0"
!pip install -q rotary-embedding-torch
!pip install -q pyyaml tqdm
!pip install -q nest-asyncio
!pip install -q pyngrok  # Ngrok for Kaggle

# Optional: SSR Eval
print("📥 Installing optional packages...")
!pip install -q ssr-eval 2>/dev/null || echo "⚠️ SSR Eval installation skipped (optional)"

print("\n" + "="*60)
print("✅ Installation complete! Verifying...")
print("="*60)

import torch
import lightning
import gradio as gr
import nest_asyncio

# Fix event loop issue
nest_asyncio.apply()

print(f"\n✓ PyTorch: {torch.__version__}")
print(f"✓ Lightning: {lightning.__version__}")
print(f"✓ Gradio: {gr.__version__}")
print(f"✓ CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✓ CUDA Version: {torch.version.cuda}")
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# Check torchvision
try:
    import torchvision
    print(f"✓ TorchVision: {torchvision.__version__}")
except Exception as e:
    print(f"⚠️ TorchVision: {str(e)[:50]}... (non-critical)")

# SSR Eval check
try:
    import ssr_eval
    print(f"✓ SSR Eval: Installed")
except ImportError:
    print(f"⚠️ SSR Eval: Not installed (optional)")

print("\n🎉 All libraries successfully installed!")
print("✅ Using Kaggle's pre-installed PyTorch (compatible with system)\n")

## 📥 2. Download Model Files

We'll download two model checkpoints:
- **One-split (0.0-1.0)**: Full time range (~1.5GB)
- **Two-split (0.5-1.0)**: Second time range (~1.5GB)

**Total download: ~3GB. This will take 5-10 minutes.**

In [ ]:
# Create checkpoint directory
!mkdir -p ckpt
print("✓ Checkpoint directory created")

In [ ]:
import os
from tqdm import tqdm

print("📥 Downloading model checkpoints...\n")
print("⏱️  This will take 5-10 minutes depending on your connection.\n")

# Model files
models = {
    'onesplit': {
        'path': 'ckpt/A2SB_onesplit_0.0_1.0_release.ckpt',
        'url': 'https://huggingface.co/nvidia/audio_to_audio_schrodinger_bridge/resolve/main/ckpt/A2SB_onesplit_0.0_1.0_release.ckpt'
    },
    'twosplit': {
        'path': 'ckpt/A2SB_twosplit_0.5_1.0_release.ckpt',
        'url': 'https://huggingface.co/nvidia/audio_to_audio_schrodinger_bridge/resolve/main/ckpt/A2SB_twosplit_0.5_1.0_release.ckpt'
    }
}

# Check and download each model
for name, info in models.items():
    if os.path.exists(info['path']):
        size_mb = os.path.getsize(info['path']) / (1024 * 1024)
        print(f"✓ {name} model already exists ({size_mb:.2f} MB)")
    else:
        print(f"⬇️  Downloading {name} model (~1.5GB)...")
        !wget -q --show-progress -O {info['path']} {info['url']}
        if os.path.exists(info['path']):
            size_mb = os.path.getsize(info['path']) / (1024 * 1024)
            print(f"✅ {name} model downloaded ({size_mb:.2f} MB)")
        else:
            print(f"❌ Failed to download {name} model!")
            print(f"Please check your internet connection and try again.")

print("\n" + "="*50)
print("✅ Model download complete!")
print("="*50)

## ⚙️ 3. Configuration

Update the configuration file with the correct model paths.

In [ ]:
import yaml

print("⚙️  Updating configuration...\n")

# Update config file
config_path = 'configs/ensemble_2split_sampling.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

config['model']['pretrained_checkpoints'] = [
    'ckpt/A2SB_onesplit_0.0_1.0_release.ckpt',
    'ckpt/A2SB_twosplit_0.5_1.0_release.ckpt'
]

with open(config_path, 'w') as f:
    yaml.dump(config, f)

print("✅ Configuration updated successfully!")
print(f"\nModel paths:")
for i, path in enumerate(config['model']['pretrained_checkpoints'], 1):
    print(f"  {i}. {path}")

## 🎨 4. Launch Gradio Web Interface

### 🚀 Ready to restore audio!

**Features:**
- 📤 Drag-and-drop file upload
- 🎤 Microphone recording
- ⚙️ Advanced settings (sampling steps, cutoff frequency)
- 📊 Real-time progress tracking
- 🔊 Instant playback and comparison
- 📈 Spectral analysis visualization

**How to use:**
1. **Run the cell below** - Wait for the Gradio link to appear
2. **Click the public link** (usually ends with `.gradio.live`)
3. **Upload audio** or record from microphone
4. **Choose mode:**
   - **Bandwidth Extension**: Restore high frequencies (for low-quality MP3s)
   - **Inpainting**: Fill in missing audio segments
5. **Adjust settings** (optional):
   - Sampling Steps: 25-100 (higher = better quality, slower)
   - Auto Cutoff: Automatically detect cutoff frequency
   - Inpainting Length: 0.1-1.0 seconds
6. **Click "🚀 Restore"** and wait for processing
7. **Listen & Download** the restored audio

**Tips:**
- Start with default settings (50 steps, auto cutoff)
- For faster results: 25-30 steps
- For best quality: 75-100 steps
- Processing time: ~2-3 minutes per 10 seconds of audio (on P100)

**⚠️ Important:**
- Keep this notebook tab open during processing
- Don't close the Kaggle session
- If you get "Out of Memory" error, reduce sampling steps or audio length
- Gradio will create a public URL that expires after 72 hours

In [ ]:
# Launch Gradio interface with public URL
print("🚀 Launching Gradio interface...\n")
print("⏱️  Please wait for the link to appear below.\n")
print("="*60)

# Kaggle için özel gradio_app kullan (ngrok ile)
import os
if os.path.exists('gradio_app_kaggle.py'):
    print("✓ Using Kaggle-optimized Gradio app with ngrok")
    !python gradio_app_kaggle.py --ngrok
else:
    print("⚠️ Using default Gradio app")
    !python gradio_app.py --share

print("\n" + "="*60)
print("✅ Gradio interface launched!")
print("Click the public link above to access the web interface.")
print("The link will be valid for 72 hours.")
print("="*60)

## 📚 5. Tips and Troubleshooting

### ⚡ Performance Optimization

**GPU Requirements:**
- ✅ **Kaggle P100**: 16GB VRAM (Recommended)
- ✅ **Kaggle T4 x2**: 2x 16GB VRAM (Excellent)
- ⚠️ **Kaggle T4**: 16GB VRAM (May work)

**Processing Times (on P100 GPU):**
- 10 seconds audio, 50 steps: ~2-3 minutes
- 30 seconds audio, 50 steps: ~5-7 minutes
- 60 seconds audio, 50 steps: ~10-15 minutes

### 🎯 Quality Settings

**Sampling Steps:**
- **25-30:** Fast (good quality)
- **50-75:** Balanced (excellent quality) ⭐ Recommended
- **75-100:** Best (outstanding quality)

**Cutoff Frequency (Bandwidth Extension):**
- **Auto-detect**: Usually best ⭐ Recommended
- **Manual adjustment:**
  - Low-quality MP3: 2000-4000 Hz
  - Medium quality: 4000-8000 Hz
  - High quality: 8000+ Hz

**Inpainting Length:**
- 0.1-0.3s: Small gaps or clicks
- 0.3-0.5s: Medium gaps
- 0.5-1.0s: Large missing segments

### 🔧 Troubleshooting

#### ❌ CUDA Out of Memory Error

**Solutions:**
1. **Reduce sampling steps** to 25-30
2. **Split audio** into shorter segments (10-20 seconds)
3. **Restart kernel**: Kernel > Restart Kernel
4. **Clear GPU memory**: Run the cell below
5. **Enable P100 GPU** in Kaggle settings

```python
# Clear GPU memory
import gc
import torch
gc.collect()
torch.cuda.empty_cache()
print("✅ GPU memory cleared")
```

#### ❌ Import Errors

**Solution:** Re-run the installation cell (Section 1, second cell)
- The notebook uses Kaggle's pre-installed PyTorch
- Only installs missing packages to avoid conflicts

#### ❌ Model Not Found Error

**Solutions:**
1. Re-run the model download cells (Section 2)
2. Check your internet connection
3. Verify files exist:
```python
!ls -lh ckpt/
```

#### ❌ Gradio Interface Not Loading

**Solutions:**
1. Wait 30-60 seconds for the link to appear
2. Check if the cell is still running
3. Restart kernel and run all cells again
4. Make sure you're using `--share` flag for public URL

#### ❌ Audio Format Error

**Solution:** Convert to WAV format
```python
import librosa
import soundfile as sf

# Convert any audio to WAV
y, sr = librosa.load('input.mp3', sr=44100)
sf.write('input.wav', y, sr)
```

#### ⚠️ Session Timeout

**Solutions:**
1. Kaggle provides 12 hours of continuous runtime
2. Keep the tab active
3. Process shorter audio files
4. Save intermediate results to `/kaggle/working/`

### 💡 Best Practices

1. **Start small**: Test with 10-20 second clips first
2. **Use defaults**: 50 steps, auto cutoff works well
3. **Monitor GPU**: Check `nvidia-smi` if issues occur
4. **Save outputs**: Download restored audio immediately
5. **Batch processing**: Process multiple files one at a time
6. **Use Kaggle Datasets**: Upload your audio files as a Kaggle dataset for easier access

### 📁 Kaggle File System

**Important directories:**
- `/kaggle/input/`: Read-only input data (datasets)
- `/kaggle/working/`: Writable directory (your work)
- `/kaggle/temp/`: Temporary files

**Upload audio files:**
1. Create a Kaggle dataset with your audio files
2. Add the dataset to your notebook
3. Access files from `/kaggle/input/your-dataset-name/`

**Download results:**
- Restored audio files are saved in `/kaggle/working/`
- Click the "Output" tab to download files
- Or use the Gradio interface to download directly

### 📖 License and Usage

- **Model:** NVIDIA OneWay NonCommercial License
- **Code:** NVIDIA Source Code License - Non Commercial
- **Commercial Use:** Contact NVIDIA for licensing
- **Research Use:** Free for academic and research purposes

### 🔗 Additional Resources

- **Paper:** [arXiv:2501.11311](https://arxiv.org/abs/2501.11311)
- **GitHub:** [test4373/diffusion-audio-restoration](https://github.com/test4373/diffusion-audio-restoration-colab-Kaggle-.git)
- **Original NVIDIA Repo:** [NVIDIA/diffusion-audio-restoration](https://github.com/NVIDIA/diffusion-audio-restoration)
- **Demo:** [NVIDIA Research](https://research.nvidia.com/labs/adlr/A2SB/)
- **Models:** [HuggingFace](https://huggingface.co/nvidia/audio_to_audio_schrodinger_bridge)

### 📧 Support

- **Issues:** [GitHub Issues](https://github.com/test4373/diffusion-audio-restoration-colab-Kaggle-/issues)
- **Original NVIDIA Issues:** [NVIDIA GitHub](https://github.com/NVIDIA/diffusion-audio-restoration/issues)
- **Kaggle Community:** [Kaggle Forums](https://www.kaggle.com/discussions)

---

### 🎉 Thank You!

Thank you for using this notebook on Kaggle!

**Citation:**
```bibtex
@article{kong2025a2sb,
  title={A2SB: Audio-to-Audio Schrodinger Bridges},
  author={Kong, Zhifeng and Shih, Kevin J and Nie, Weili and Vahdat, Arash and Lee, Sang-gil and Santos, Joao Felipe and Jukic, Ante and Valle, Rafael and Catanzaro, Bryan},
  journal={arXiv preprint arXiv:2501.11311},
  year={2025}
}
```

### ⭐ Support This Project

If you find this project useful:
- ⭐ Star the [GitHub repository](https://github.com/test4373/diffusion-audio-restoration-colab-Kaggle-.git)
- 🐛 Report bugs or suggest features
- 📢 Share with others who might benefit
- 👍 Upvote this Kaggle notebook

---

**Made with ❤️ for the audio restoration community**

**Optimized for Kaggle with GPU memory management and user-friendly interface**

### 🆓 Why Kaggle?

Kaggle offers several advantages:
- **Free GPU access**: 30 hours per week
- **Powerful hardware**: P100 GPU with 16GB VRAM
- **Generous resources**: 30GB RAM, 73GB disk
- **Persistent storage**: Save your work and models
- **Community**: Share and collaborate with others
- **No subscription**: Completely free!

Perfect for audio restoration projects! 🎵